In [2]:
import sys
sys.path.append("..")
from utils.SolutionChecker import Case, check_solution

# Braket Expansion

### Problem Description

Implement a similar behavior to bash's brace expansion behavior as a runnable program.

For a valid input, print the output. For an invalid input, print nothing and exit.

### Considerations

  - Any input without properly matching braces is invalid
  - Commas should only appear within braces. 
  - Restrict the input character set to \[a-zA-Z\{\},\] 
  - Braces should not be empty, and there should be no "empty" options within braces i.e. {A,}

### Examples of Valid Input
  - {A,B,C} -> A B C
  - AB{C,D} -> ABC ABD
  - {A,B}{C,D} -> AC AD BC BD
  - {A,B{C,D}} -> A BC BD
  - {{A},{B}} -> A B
  - {ABC} -> ABC
  - ABC -> ABC

###  Examples of invalid input
  - }ABC
  - {ABC
  - }{
  - {}
  - A,B,C
  - A B C
  - {A{B,C}
  - {A,}


In [3]:
cases = [Case('{A,B,C}','A B C'), 
         Case('AB{C,D}', 'ABC ABD'),
         Case('{A,B}{C,D}', 'AC AD BC BD'),
         Case('{A,B{C,D}}', 'A BC BD'), 
         Case('{{A},{B}}','A B'),
         Case('{ABC}','ABC'),
         Case('{ABC',None),
         Case('{A,B,C}{A,B{{A}}}','AA ABA BA BBA CA CBA'), 
         Case('D{A,B}{A,B{{C}}}','DAA DABC DBA DBBC'), 
         Case('AB{C,D}', 'ABC ABD'),
         Case('{A}{B}{C,D}', 'ABC ABD'),
         Case('A,B{C,D}', None), 
         Case('{{A},{B}}','A B'),
         Case('{ABC}','ABC'),
         Case('}ABC',None),
         Case('}{',None),
         Case('{}',None),
         Case('A,B,C',None),
         Case('{A{B,C}',None),
         Case('{A,',None),
         Case('A{?}',None),
         Case('',None),
        ]

In [5]:
def merge_values(current : list, returned : list):
    """Merge two lists of expression values."""
    
    result = []
    if returned:
        if not current:
            return returned
        
        for current_val in current:
            
            for returned_val in returned:
                result.append(f'{current_val}{returned_val}')
    else:
        return current
        
    return result

def expand_bracket(input_values : list, searching : list = None):
    current= []
    result = []
    if searching is None:
        searching = []
    
    while input_values:
        
        #Pop the list as a queue to get the next character to process
        val = input_values.pop(0)
        
        if val == '{':
            if searching and searching[-1] == ',':
                searching.pop(-1)
                
            searching.append('}')
            
            returned = expand_bracket(input_values, searching)
            current = merge_values(current,returned)
            
        elif val == '}':
            if searching and searching[-1] == '}':
                    searching.pop(-1)
            else:
                raise ValueError(f'Invalid input - The character "{val}" was not expected.')
                
            result += current
            
            return result
        
        elif val == ',':
            if '}' not in searching:
                raise ValueError(f'Invalid input - The character "{val}" is not within {{}}')
            
            searching.append(',')
            result += current
            current.clear()
            
        elif val.isalpha():
            
            if searching and searching[-1] == ',':
                searching.pop(-1)
            
            if not current:
                current.append(val)
            else:
                current[-1] += val
        else:
            raise ValueError(f'Invalid input - The character "{val}" does not match [a-zA-Z,{{}}].')
    
        last_char = val
    
    if searching:
        raise ValueError(f'Invalid input - Expected character(s) {searching}.')
    
    result += current
    
    if not result:
        raise ValueError(f'Invalid input - Expressions cannot be empty.')
    
    return result

def apply_expand_bracket(input_string):
    
    try:
        result = expand_bracket(list(input_string))
        result = ' '.join(result)
    except ValueError as e:
        result = None
        # print(e)
        
    return result    

check_solution(cases, apply_expand_bracket)

Case # 0 | Input: {A,B,C} | A B != A B C | Runtime: 7.62939453125e-06 
Case # 1 | Input: AB{C,D} | ABC != ABC ABD | Runtime: 6.198883056640625e-06 
Case # 2 | Input: {A,B}{C,D} | AC != AC AD BC BD | Runtime: 1.0251998901367188e-05 
Case # 3 | Input: {A,B{C,D}} | A != A BC BD | Runtime: 6.4373016357421875e-06 
Case # 4 | Input: {{A},{B}} | None != A B | Runtime: 5.4836273193359375e-06 
Case # 5 | Input: {ABC} | None != ABC | Runtime: 3.337860107421875e-06 
Case # 6 | Input: {ABC | None == None | Runtime: 5.0067901611328125e-06 
Case # 7 | Input: {A,B,C}{A,B{{A}}} | AA BA != AA ABA BA BBA CA CBA | Runtime: 7.367134094238281e-05 
Case # 8 | Input: D{A,B}{A,B{{C}}} | DAA != DAA DABC DBA DBBC | Runtime: 8.344650268554688e-06 
Case # 9 | Input: AB{C,D} | ABC != ABC ABD | Runtime: 3.814697265625e-06 
Case # 10 | Input: {A}{B}{C,D} | C != ABC ABD | Runtime: 5.0067901611328125e-06 
Case # 11 | Input: A,B{C,D} | None == None | Runtime: 2.384185791015625e-06 
Case # 12 | Input: {{A},{B}} | None !